In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import os

In [10]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [11]:
# === 1. Image Transforms ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagenet normalization
                         std=[0.229, 0.224, 0.225])
])


In [12]:

# === 2. Load Dataset ===
train_dataset = datasets.ImageFolder("cats_and_dogs/train", transform=transform)
val_dataset = datasets.ImageFolder("cats_and_dogs/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [13]:
# === 3. Load Pretrained ResNet101 ===
model = models.resnet101(pretrained=True)


In [14]:
# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier for 2 classes
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 1),             # Output = 1 score
    nn.Sigmoid()                   # For binary classification
)

model = model.to(device)


In [15]:
# === 4. Define Loss and Optimizer ===
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)


In [16]:

# === 5. Training Loop ===
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")




Epoch [1/5], Loss: 27.2567, Accuracy: 95.85%
Epoch [2/5], Loss: 16.2145, Accuracy: 97.61%
Epoch [3/5], Loss: 14.6272, Accuracy: 97.84%
Epoch [4/5], Loss: 14.9660, Accuracy: 97.85%
Epoch [5/5], Loss: 13.5271, Accuracy: 98.04%


In [17]:
# === 6. Save the model ===
torch.save(model.state_dict(), "resnet101_cat_dog.pth")
print("Model saved!")

Model saved!
